In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import sys  # For simplicity, we'll read config file from 1st CLI param sys.argv[1]
import json
import logging
import requests
import msal
import pandas as pd
from datetime import date, timedelta,datetime
from delta.tables import *
from pyspark.sql.functions import count_distinct,col,year,month
from pyspark.sql import functions as F

spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")


StatementMeta(, 13edf138-68be-4675-a60b-fc9d09db9243, 5, Finished, Available)

In [2]:
def createDeltaTables(tableName,sdf):
    from pyspark.sql.functions import col,year,month
    from pyspark.sql import functions as F
    from datetime import datetime
    sdf = sdf.withColumn('Year', F.lit(datetime.now().strftime("%Y")))
    sdf = sdf.withColumn('Month', F.lit(datetime.now().strftime("%m")))
    sdf = sdf.withColumn('RunDate', F.lit(datetime.now().strftime("%Y-%m-%dT%H:%M:%S")))
    sdf.write.mode("append").option("mergeSchema","true").format("parquet").save("Files/Stage/"+ datetime.now().strftime("%Y/%m/%d") + "/" + tableName)
    return 1

StatementMeta(, 1e8b5a59-5963-4148-a31b-333453510abf, 4, Finished, Available)

In [2]:
def createDeltaTablesV1(tableName,key):
    from pyspark.sql.functions import col,year,month
    from pyspark.sql import functions as F

    vDate=date.today().strftime("%Y/%m/%d")

    if DeltaTable.isDeltaTable(spark, "/2fe98f6a-0a9a-4a7c-bb43-37c1cbc58fda/Tables/"+tableName):
        toUpdatePath="/2fe98f6a-0a9a-4a7c-bb43-37c1cbc58fda/Tables/" + tableName + "/"
        sourcePath="Files/Stage/" + vDate + "/" + tableName
        #sourcePath="Files/Stage/2023/07/22/" +  tableName
        sdfToUpdate = DeltaTable.forPath(spark,toUpdatePath)
        sdfSource= spark.read.format("parquet").load(sourcePath)
        
        if tableName=="Lakehouses" or tableName=="Warehouses" or tableName=="Synapsenotebooks":
            sdfSource = sdfSource.filter(sdfSource.id!="NaN")            

        sdfSource=sdfSource.drop("RunDate")
        sdfSource=sdfSource.distinct()
        sdfSource=sdfSource.withColumn('RunDate', F.lit(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))) 

        (sdfToUpdate.alias("t")
        .merge(sdfSource.alias("s"), key)
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
        )
    else: 
           # sourcePath="Files/Stage/2023/07/21/" + tableName 
            sourcePath="Files/Stage/" + datetime.now().strftime("%Y/%m/%d") + "/" + tableName 
            sdf = spark.read.format("parquet").load(sourcePath)
            sdf.write.mode("overwrite").option("mergeSchema","true").format("delta").save("Tables/" + tableName)
    return 1

StatementMeta(, 13edf138-68be-4675-a60b-fc9d09db9243, 4, Finished, Available)

In [4]:
from datetime import datetime, timedelta
url=None

if DeltaTable.isDeltaTable(spark, "/2fe98f6a-0a9a-4a7c-bb43-37c1cbc58fda/Tables/Workspaces"):
    RunDate=spark.sql("select max(RunDate) from Workspaces").first()["max(RunDate)"]
    Begindate = datetime.strptime(RunDate, "%Y-%m-%dT%H:%M:%S") 
    # Begin Date -5 date to go back 5 days. 
    Begindate = Begindate + timedelta(days=-2)
    print(Begindate)
    Begindate=Begindate.strftime("%Y-%m-%dT%H:%M:%S.%f")
    print(Begindate)
    url = "https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified?modifiedSince=" + Begindate +"0Z"
    print(url) 

StatementMeta(, 1e8b5a59-5963-4148-a31b-333453510abf, 6, Finished, Available)

2023-07-26 15:37:21
2023-07-26T15:37:21.000000
https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified?modifiedSince=2023-07-26T15:37:21.0000000Z


In [5]:
def getToken():
    import requests
    import msal
    config = {
                "authority": "https://login.microsoftonline.com/ReplaceWithYourTenant",
                "client_id": "ReplaceWithYourClientId",
                "scope": ["https://analysis.windows.net/powerbi/api/.default"],
                "secret": "ReplaceWithYourClient'sSceret",
                "endpoint": "https://api.powerbi.com/v1.0/myorg/admin/activityevents"
            }

#Get yesterdays date and convert to string


# Create a preferably long-lived app instance which maintains a token cache.
    app = msal.ConfidentialClientApplication(
            config["client_id"], authority=config["authority"],
            client_credential=config["secret"],
            )

        # The pattern to acquire a token looks like this.
    result = None

        # Firstly, looks up a token from cache
        # Since we are looking for token for the current app, NOT for an end user,
        # notice we give account parameter as None.
    result = app.acquire_token_silent(config["scope"], account=None)

    if not result:
        logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
        result = app.acquire_token_for_client(scopes=config["scope"])

    if "access_token" in result:
            # Calling graph using the access token
        logging.info("Got the Token")

    else:
        print(result.get("error"))
        print(result.get("error_description"))
        print(result.get("correlation_id")) 
    
    return result

result=getToken()

StatementMeta(, 1e8b5a59-5963-4148-a31b-333453510abf, 7, Finished, Available)

In [6]:
#print(result)
activityDate = date.today() 

#url=None
#Set Power BI REST API to get Activities for today
if not url:
    #url = "https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified?excludePersonalWorkspaces=True" 
    url = "https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified" 

print(url)

access_token = result['access_token']
header = {'Content-Type':'application/json', 'Authorization':f'Bearer {access_token}'}
response = requests.get(url=url, headers=header)

if response.status_code ==200 :
    response_obj = response.json()
    #print(response_obj)


StatementMeta(, 1e8b5a59-5963-4148-a31b-333453510abf, 8, Finished, Available)

https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified?modifiedSince=2023-07-26T15:37:21.0000000Z


In [7]:
#response.status_code
#print(response.text)
print(f"Total Workspaces to iterate {len(response_obj)}")
#print(response_obj)

StatementMeta(, 1e8b5a59-5963-4148-a31b-333453510abf, 9, Finished, Available)

Total Workspaces to iterate 1530


In [ ]:
# Printing total workspaces to iterate 
from pyspark.sql.types import *

print(f"Total Workspaces to iterate {len(response_obj)}")

from time import sleep 
for i in range(0,len(response_obj),100): #for i in range(0,100,100):
    print(f"Iterating workpace Number {(i+100)}")
    workspaces=response_obj[i:i+100]
    workspaceList = [] 
    for item in workspaces:
        workspaceList.append(item['id'])
        WorkspaceListToSend= {"workspaces":workspaceList}
        #print(WorkspaceListToSend)
    ################# Sending the pst request to get workspaces details #####################################################################    
    url = "https://api.powerbi.com/v1.0/myorg/admin/workspaces/getInfo?lineage=true&datasourceDetails=true&getArtifactUsers=true" 
    
    result=getToken()
    access_token = result['access_token']
    header = {'Content-Type':'application/json', 'Authorization':f'Bearer {access_token}'}

    scan = requests.post(url,headers=header,json=WorkspaceListToSend)
    print(scan.status_code)
    sleep(0.2)
    scanResponse=json.loads(scan.text)
    print(scanResponse)

    waitForScanToComplete =True
    #print(scan.id) 
    scanId = scanResponse['id']
    #####################Checking to see if the Scan completed, at this time not coding for any other status than Succeded, but may have to look at that as well. 
    while waitForScanToComplete:
        url="https://api.powerbi.com/v1.0/myorg/admin/workspaces/scanStatus/" + scanId
        status = requests.get(url,headers=header) 
        statusResponse=json.loads(status.text)
        print(statusResponse)
        statusState = statusResponse['status']
        print(statusState)
        if statusState=="Succeeded":
            break 
        sleep(0.2)
    # The status has changed now to succeeded, get the result
    url = "https://api.powerbi.com/v1.0/myorg/admin/workspaces/scanResult/" + scanId
    scanResult = requests.get(url,headers=header)

    ################################# Converting Text response to json object and creating pandas dataframe ############################################
    print(scanResult)
    scanResultResponse=scanResult.text
    #print(scanResultResponse)
    print(type(scanResultResponse))
    data = json.loads(scanResultResponse)
    df=pd.DataFrame(data['workspaces'])

    lakehouse = pd.DataFrame()
    warehouses = pd.DataFrame()
    synapsenotebook = pd.DataFrame()
    reports = pd.DataFrame()
    reportUsers=pd.DataFrame()
    dashboards = pd.DataFrame()
    dashboardUsers=pd.DataFrame()
    datasets = pd.DataFrame()
    datasetUsers=pd.DataFrame()
    dataflows = pd.DataFrame()
    dataflowUsers=pd.DataFrame()
    workspaceUsers = pd.DataFrame()
    dfdatasourceInstances = pd.DataFrame()
    
    if 'users' in df.columns:
       try:
            workspaceUsers = pd.json_normalize(data=data['workspaces'],record_path=['users'],record_prefix='users_',meta="id")
       except:
            pass
    
    try:
        if 'datasourceInstances' in data:
            dfdatasourceInstances=pd.DataFrame(data['datasourceInstances'])
            datasourceInstances_schema=StructType([StructField("datasourceType",StringType(),True),
                StructField("connectionDetails",StringType(),True),
                StructField("datasourceId",StringType(),True),
                StructField("gatewayId",StringType(),True)
                ]) 
    except:
            pass
    ##################################################################################################################################

    if 'reports' in df.columns:
        try:
            reports = pd.json_normalize(data=data['workspaces'],record_path=['reports'],record_prefix='reports_',meta="id")
            if 'reports_users' in reports.columns:
                reportUsers=reports[['reports_id','reports_users']]
                reportUsers=pd.DataFrame(reportUsers).explode("reports_users")
                reportUsers=pd.concat([reportUsers,reportUsers.pop("reports_users").apply(pd.Series)],axis=1)
        except:
            pass

    if 'dashboards' in df.columns:
        try:
            dashboards = pd.json_normalize(data=data['workspaces'],record_path=['dashboards'],record_prefix='dashboards_',meta="id")
            if 'dashboards_users' in dashboards.columns:
                dashboardUsers=dashboards[['dashboards_id','dashboards_users']]
                dashboardUsers=pd.DataFrame(dashboardUsers).explode("dashboards_users")
                dashboardUsers=pd.concat([dashboardUsers,dashboardUsers.pop("dashboards_users").apply(pd.Series)],axis=1)
        except:
            pass
    
    if 'datasets' in df.columns:
        try:
            datasets = pd.json_normalize(data=data['workspaces'],record_path=['datasets'],record_prefix='datasets_',meta="id")
            if 'datasets_users' in datasets.columns:
                datasetUsers=datasets[['datasets_id','datasets_users']]
                datasetUsers=pd.DataFrame(datasetUsers).explode("datasets_users")
                datasetUsers=pd.concat([datasetUsers,datasetUsers.pop("datasets_users").apply(pd.Series)],axis=1)
        except:
            pass
    
    if 'dataflows' in df.columns:
        try:
            dataflows = pd.json_normalize(data=data['workspaces'],record_path=['dataflows'],record_prefix='dataflows_',meta="id")
            if 'dataflows_users' in dataflows.columns:
                dataflowUsers=dataflows[['dataflows_objectId','dataflows_users']]
                dataflowUsers=pd.DataFrame(dataflowUsers).explode("dataflows_users")
                dataflowUsers=pd.concat([dataflowUsers,dataflowUsers.pop("dataflows_users").apply(pd.Series)],axis=1)
        except:
            pass

    
    ################################# Define the structure for the Spark Tables ############################################
   
   
    if not reports.empty:
        reports_schema = StructType()
        for column in reports:
            reports_schema.add(str(column),StringType())

    if not reportUsers.empty:
        reportUsers_schema = StructType()
        for column in reportUsers:
            reportUsers_schema.add(str(column),StringType())
    

    if not dashboards.empty:
        dashboards_schema = StructType()
        for column in dashboards:
            dashboards_schema.add(str(column),StringType())
     
    if not dashboardUsers.empty:
        dashboardUsers_schema = StructType()
        for column in dashboardUsers:
            dashboardUsers_schema.add(str(column),StringType())
     
    if not datasets.empty:
        datasets_schema = StructType()
        for column in datasets:
            datasets_schema.add(str(column),StringType())
    
    if not datasetUsers.empty:
        datasetUsers_schema = StructType()
        for column in datasetUsers:
            datasetUsers_schema.add(str(column),StringType())
    
    if not dataflows.empty:
        dataflow_schema= StructType()
        for column in dataflows:
            dataflow_schema.add(str(column),StringType())
    
    if not dataflowUsers.empty:
        dataflowUsers_schema= StructType()
        for column in dataflowUsers:
            dataflowUsers_schema.add(str(column),StringType())
    
    if not workspaceUsers.empty:
        workspaceUsers_schema= StructType()
        for column in workspaceUsers:
            workspaceUsers_schema.add(str(column),StringType())

    # May be we should drop these columns ,'lakehouse','warehouses','synapsenotebook'
    workspaces=df.drop(['reports','dashboards','datasets','dataflows','datamarts','users'],axis=1,errors='ignore')

    try:
        if 'lakehouse' in df.columns:
            lakehouse=workspaces[['id','lakehouse']]
            lakehouse.rename(columns={"id":"workspaceid"},inplace=True)
            lakehouse=pd.DataFrame(lakehouse).explode("lakehouse")
            lakehouse=pd.concat([lakehouse,lakehouse.pop("lakehouse").apply(pd.Series)],axis=1).drop(columns=0)
    except:
            pass

    try:
        if 'warehouses' in df.columns:
            warehouses=workspaces[['id','warehouses']]
            warehouses.rename(columns={"id":"workspaceid"},inplace=True)
            warehouses=pd.DataFrame(warehouses).explode("warehouses")
            warehouses=pd.concat([warehouses,warehouses.pop("warehouses").apply(pd.Series)],axis=1).drop(columns=0)
    except:
            pass

    try:
        if 'synapsenotebook' in df.columns:
            synapsenotebook=workspaces[['id','synapsenotebook']]
            synapsenotebook.rename(columns={"id":"workspaceid"},inplace=True)
            synapsenotebook=pd.DataFrame(synapsenotebook).explode("synapsenotebook")
            synapsenotebook=pd.concat([synapsenotebook,synapsenotebook.pop("synapsenotebook").apply(pd.Series)],axis=1).drop(columns=0)
    except:
            pass

    if not workspaces.empty:
            workspace_schema= StructType()
            for column in workspaces:
                workspace_schema.add(str(column),StringType())

    if not lakehouse.empty:
            lakehouse_schema= StructType()
            for column in lakehouse:
                lakehouse_schema.add(str(column),StringType())
        
    if not warehouses.empty:
            warehouses_schema= StructType()
            for column in warehouses:
                warehouses_schema.add(str(column),StringType())
    
    if not synapsenotebook.empty:
        synapsenotebook_schema= StructType()
        for column in synapsenotebook:
            synapsenotebook_schema.add(str(column),StringType())

    ############################################ Create Spark dataframe and Tables #################################################
    
    if not reports.empty:
        reportsdf = spark.createDataFrame(reports,schema=reports_schema)
        createDeltaTables('Reports',reportsdf)

    if not reportUsers.empty:
        reportUsersdf = spark.createDataFrame(reportUsers,schema=reportUsers_schema)
        createDeltaTables('ReportUsers',reportUsersdf)

    if not dashboards.empty:
        dashboardsdf=spark.createDataFrame(dashboards,dashboards_schema)
        createDeltaTables('Dashboards',dashboardsdf)
    
    if not dashboardUsers.empty:
        dashboardUsersdf=spark.createDataFrame(dashboardUsers,dashboardUsers_schema)
        createDeltaTables('DashboardUsers',dashboardUsersdf)
    
    if not datasets.empty:
        datasetsdf=spark.createDataFrame(datasets,datasets_schema)
        createDeltaTables('Datasets',datasetsdf)
    
    if not datasetUsers.empty:
        datasetUsersdf=spark.createDataFrame(datasetUsers,datasetUsers_schema)
        createDeltaTables('DatasetUsers',datasetUsersdf)

    if not dataflows.empty:
        dataflowsdf=spark.createDataFrame(dataflows,dataflow_schema)
        createDeltaTables('Dataflows',dataflowsdf)

    if not dataflowUsers.empty:
        dataflowUsersdf=spark.createDataFrame(dataflowUsers,dataflowUsers_schema)
        createDeltaTables('DataflowUsers',dataflowUsersdf)

    if not lakehouse.empty:
        #if "id" in lakehouse.columns:
         #   lakehouse = lakehouse.filter(lakehouse.id!="NaN")
        lakehousedf=spark.createDataFrame(lakehouse,lakehouse_schema)
        createDeltaTables('Lakehouses',lakehousedf)

    if not warehouses.empty:
        #if "id" in warehouses.columns:
         #   warehouses = warehouses.filter(warehouses.id!="NaN")
        warehousesdf=spark.createDataFrame(warehouses,warehouses_schema)
        createDeltaTables('Warehouses',warehousesdf)
    
    if not synapsenotebook.empty:
        #if "id" in synapsenotebook.columns:
         #   synapsenotebook = synapsenotebook.filter(synapsenotebook.id!="NaN")
        synapsenotebookdf=spark.createDataFrame(synapsenotebook,synapsenotebook_schema)
        createDeltaTables('Synapsenotebooks',synapsenotebookdf)

    #datamartsdf = spark.createDataFrame(datamarts)
    if not workspaceUsers.empty:
        workspaceUsersdf =spark.createDataFrame(workspaceUsers,workspaceUsers_schema)
        createDeltaTables('WorkspaceUsers',workspaceUsersdf)
    
    if not dfdatasourceInstances.empty:
        datasourceInstaces =spark.createDataFrame(dfdatasourceInstances,datasourceInstances_schema)
        createDeltaTables('datasourceInstances',datasourceInstaces)
    
    workspacedf=spark.createDataFrame(workspaces,workspace_schema)
    createDeltaTables('Workspaces',workspacedf)

     #createDeltaTables('datamarts',datamartsdf)   


In [9]:
#if not reports.empty:
Key="t.reports_id = s.reports_id"
createDeltaTablesV1('Reports',Key)

try:
    Key="t.reports_id = s.reports_id AND t.graphId=s.graphId AND t.reportUserAccessRight=s.reportUserAccessRight"
    createDeltaTablesV1('ReportUsers',Key)
except:
    pass
#if not reportUsers.empty:


#if not dashboards.empty:
key="t.dashboards_id = s.dashboards_id"
createDeltaTablesV1('Dashboards',key)

try:
    Key="t.dashboards_id = s.dashboards_id AND t.graphId=s.graphId AND s.dashboardUserAccessRight=t.dashboardUserAccessRight"
    createDeltaTablesV1('DashboardUsers',key) 
except:
    vDate=date.today().strftime("%Y/%m/%d")
    toUpdatePath="/2fe98f6a-0a9a-4a7c-bb43-37c1cbc58fda/Tables/" + "DashboardUsers" + "/"
    sourcePath="Files/Stage/" + vDate + "/" + "DashboardUsers"
        #sourcePath="Files/Stage/2023/07/22/" +  tableName
    sdfToUpdate = DeltaTable.forPath(spark,toUpdatePath)
    sdfSource= spark.read.format("parquet").load(sourcePath)
        
    sdfSource=sdfSource.drop("RunDate")
    sdfSource=sdfSource.distinct()
    sdfSource=sdfSource.withColumn('RunDate', F.lit(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))) 

    (sdfToUpdate.alias("t")
        .merge(sdfSource.alias("s"), "t.dashboards_id = s.dashboards_id AND t.graphId=s.graphId AND s.dashboardUserAccessRight=t.dashboardUserAccessRight")
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

#if not dashboardUsers.empty:

#if not datasets.empty:
key="t.datasets_id = s.datasets_id"
createDeltaTablesV1('Datasets',key)

#if not datasetUsers.empty:
try:
    Key="t.datasets_id = s.datasets_id AND t.graphId=s.graphId AND t.datasetUserAccessRight=s.datasetUserAccessRight"
    createDeltaTablesV1('DatasetUsers',key) 
except:
    vDate=date.today().strftime("%Y/%m/%d")
    toUpdatePath="/2fe98f6a-0a9a-4a7c-bb43-37c1cbc58fda/Tables/" + "DatasetUsers" + "/"
    sourcePath="Files/Stage/" + vDate + "/" + "DatasetUsers"
        #sourcePath="Files/Stage/2023/07/22/" +  tableName
    sdfToUpdate = DeltaTable.forPath(spark,toUpdatePath)
    sdfSource= spark.read.format("parquet").load(sourcePath)
        
    sdfSource=sdfSource.drop("RunDate")
    sdfSource=sdfSource.distinct()
    sdfSource=sdfSource.withColumn('RunDate', F.lit(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))) 

    (sdfToUpdate.alias("t")
        .merge(sdfSource.alias("s"), "t.datasets_id = s.datasets_id AND t.graphId=s.graphId AND t.datasetUserAccessRight=s.datasetUserAccessRight")
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

#if not dataflows.empty:
key="t.dataflows_objectId = s.dataflows_objectId"
createDeltaTablesV1('Dataflows',key)

#if not dataflowUsers.empty:

try:
    Key="t.dataflows_objectId = s.dataflows_objectId AND t.graphId=s.graphId"
    createDeltaTablesV1('DataflowUsers',key)
except:
    vDate=date.today().strftime("%Y/%m/%d")
    toUpdatePath="/2fe98f6a-0a9a-4a7c-bb43-37c1cbc58fda/Tables/" + "DataflowUsers" + "/"
    sourcePath="Files/Stage/" + vDate + "/" + "DataflowUsers"
        #sourcePath="Files/Stage/2023/07/22/" +  tableName
    sdfToUpdate = DeltaTable.forPath(spark,toUpdatePath)
    sdfSource= spark.read.format("parquet").load(sourcePath)
        
    sdfSource=sdfSource.drop("RunDate")
    sdfSource=sdfSource.distinct()
    sdfSource=sdfSource.withColumn('RunDate', F.lit(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))) 

    (sdfToUpdate.alias("t")
        .merge(sdfSource.alias("s"), "t.dataflows_objectId = s.dataflows_objectId AND t.graphId=s.graphId")
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

#if not lakehouse.empty:
try:
    key="t.id = s.id"
    createDeltaTablesV1('Lakehouses',key)
except:
    pass

#if not warehouses.empty:
try:
    key="t.id = s.id"
    createDeltaTablesV1('Warehouses',key)
except:
    pass

#if not synapsenotebook.empty:
try:
    key="t.id = s.id"
    createDeltaTablesV1('Synapsenotebooks',key)
except:
    pass

#if not workspaceUsers.empty:

try:
    key="t.id = s.id AND t.users_graphId=s.users_graphId"
    createDeltaTablesV1('WorkspaceUsers',key)
except:
    pass


#if not dfdatasourceInstances.empty:
try:
    key="t.datasourceId = s.datasourceId AND t.gatewayId=s.gatewayId AND s.datasourceType=t.datasourceType"
    createDeltaTablesV1('datasourceInstances',key)
except:
    pass

#if not workspaces.empty:
try:
    key="t.id = s.id"
    createDeltaTablesV1('Workspaces',key)
except:
    pass

StatementMeta(, 1e8b5a59-5963-4148-a31b-333453510abf, 11, Finished, Available)

1